# Fellesordning målinger:
- Hvor mange saker går til behandling av FO = antall AFP privat per dag/uke/mnd?
- Hvor mange dager må disse sakene vente på svar gjennomsnitt?
- Hvor mange av disse sakene får resultat fra FO med de forskjellige typene: INNV, AVSLÅTT, TRUKKET, FEILREGISTRERT, Ingen resultat mottatt
- Antall kall til forskjellige tjenester? (TPEN482, TPEN124, TPEN2022) Antall feil ?
- Antall saker til manuell behandling vs automatisk behandling ( =PPEN006)

## Spørsmål:
- Hva betyr opprettet_av i T_AFP_PRIV_RES_FK, kommer ikke alt fra srvElsam_FK?

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import plotly.express as px
from datetime import datetime
from datastory import DataStory

sys.path.append('/home/jupyter/pensjon-data-analyse/lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()
df_afp_priv_res = pandas_utils.pandas_from_sql('/home/jupyter/pensjon-data-analyse/sql/afp_priv_res.sql', con, tuning=1000)
con.close()

In [ ]:
df_afp_priv_res.columns = map(str.lower, df_afp_priv_res.columns)
df = df_afp_priv_res.copy()

df = df.dropna()
df["vedtaksdato"] = pd.to_datetime(df.vedtaksdato)

df.set_index('vedtaksdato',inplace=True)

df['year']=df.index.year
df['month']=df.index.month
df['day']=df.index.day

df = df[df.year >= 2022]

In [ ]:
figs = {}
figs["daglige_vedtak"] = px.bar(df, x=df.index, y="antall", color="resultat", color_discrete_map={"INNVILGET": 'green', "FEILREG": 'orange', "AVSLATT": 'red', "TRUKKET": 'purple'})

In [ ]:
df_mnd = df.groupby("resultat")[["antall"]].resample('MS').sum().reset_index()

figs["månedlige_vedtak"] = px.bar(df_mnd, df_mnd.vedtaksdato, "antall", color="resultat", color_discrete_map={"INNVILGET": 'green', "FEILREG": 'orange', "AVSLATT": 'red', "TRUKKET": 'purple'})

# Ledetid

In [ ]:
con = pesys_utils.open_pen_connection()
df_afp_priv_ledetid = pandas_utils.pandas_from_sql('/home/jupyter/pensjon-data-analyse/sql/afp_priv_ledetid.sql', con, tuning=100000)
con.close()

In [ ]:
df_afp_priv_ledetid.columns = map(str.lower, df_afp_priv_ledetid.columns)
df = df_afp_priv_ledetid.copy().dropna()
df["vedtaksdato"] = pd.to_datetime(df.vedtaksdato)
df = df[df.vedtaksdato.dt.year >= 2022]
df.head()

In [ ]:
px.histogram(df.dager)

In [ ]:
df[df.dager < 0].reset_index()

# Datastory

In [ ]:
for key, fig in figs.items():
    fig.show()

In [ ]:
ds = DataStory("AFP Privat resultater")
ds.markdown("*Dette er ikke offentlig statistikk*. Tall hentet fra `PEN.T_AFP_PRIV_RES_FK`")
ds.markdown("Vedtak per dag")
ds.plotly(figs["daglige_vedtak"].to_json())
ds.markdown("Vedtak per måned")
ds.plotly(figs["månedlige_vedtak"].to_json())

ds.publish(url="https://datamarkedsplassen.intern.nav.no/api")